# Analysis of Adaptive Runs vs Non-Adaptive Runs

In [ ]:
import a3fe as a3 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams, rcParamsDefault
import pickle
import scipy.stats as stats
from sklearn import metrics
rcParams.update(rcParamsDefault)
plt.style.use("seaborn-v0_8-colorblind")
plt.rc('text.latex', preamble=r'\usepackage{amsmath}')
from typing import List, Tuple, Dict, Callable, Union, Optional, Any, Dict
%matplotlib inline
from scipy.stats import linregress, kruskal, t, sem, wilcoxon
from matplotlib import gridspec
import pymbar
print(pymbar.version.version)
import logging

ligs = {
    "2": "lig_2",
    "3": "lig_3",
    "4": "lig_4",
    "8": "lig_8",
    "14": "lig_14",
    "16": "lig_16",
    "27": "lig_27",
    "39": "lig_39_cry_pose",
    "40": "lig_40",
}

adaptive_paths = {
    "2": "lig_2",
    "3": "lig_3",
    "4": "lig_4",
    "8": "lig_8",
    "14": "lig_14",
    "16": "lig_16",
    "27": "lig_27",
    "39": "lig_39_cry_pose",
    "40": "lig_40",
}

non_adapt_paths = {lig: f"../non_adaptive/{adapt_path}_5000ps" for lig, adapt_path in adaptive_paths.items()}

REF_COST = 0.21 # GPU hours per ns

## Long Analyses to be Run in TMUX Sessions

## Load in the Results of the Slow Analyses

In [ ]:
file_names = [
    "comparitive_conv_data",
    "overall_costs",
    "costs",
    "equil_dgs",
    "final_dGs_all",
    "total_costs",
    "overall_results_alibay",
    "restraint_corrections",
    "restraint_dicts",
    "lam_vals",
    "sampling_times",
    "sampling_times_nonadapt",
    "dgs_conv_adaptive_nonequil",
    "dgs_conv_non_adaptive_nonequil",
]

file_paths = {name: f"final_analysis/{name}.pkl" for name in file_names}

for var, file in file_paths.items():
    with open(file, "rb") as f:
        globals()[var] = pickle.load(f)

final_dgs_all = final_dGs_all

## Outline of Notebook

- Table of results (adaptive, non-adaptive, and Alibay results)
- Statistical tests comparing results
- Equilibration analysis
- Selection of lambda windows
- Sampling time allocations + costs
- Convergence plots

## Table of Overall Results

In [ ]:
exp_dgs = pd.read_csv("final_analysis/exp_dgs.csv", index_col=1)
exp_dgs

In [ ]:
def get_95_ci(data: np.ndarray) -> Tuple[float, float]:
    """Get the 95% confidence interval for a given array of data using scipy.stats.sem"""
    mean_free_energy = np.mean(data)
    conf_int = t.interval(
        0.95,
        len(data) - 1,
        mean_free_energy,
        scale=sem(data),
    )[1] - mean_free_energy # 95 % C.I.
    return conf_int
    
symmetry_corrections = {lig: 0.0 for lig in ligs}

exp_dgs = pd.read_csv("final_analysis/exp_dgs.csv", index_col=1)


results_summary = {}
for system in final_dgs_all:
    results_summary[system] = {}
    for method in final_dgs_all[system]:
        dgs = np.array(final_dgs_all[system][method]["dgs"])
        dg_tot = np.mean(dgs) + symmetry_corrections[system]
        dg_err = get_95_ci(dgs)
        result_str = f"{dg_tot:.2f}" + r" $\pm$ " + f"{dg_err:.2f}"
        results_summary[system][method] = result_str

    # Add the Alibay value
    alibay_dg = overall_results_alibay[system]["mean_dG"]
    alibay_err = overall_results_alibay[system]["95_ci"]
    alibay_str = f"{alibay_dg:.2f}" + r" $\pm$ " + f"{alibay_err:.2f}"
    results_summary[system]["Alibay"] = alibay_str

    # Add the experimental value
    exp_dg = exp_dgs.loc[f"lig_{system}", "exp_dg"]
    exp_err = exp_dgs.loc[f"lig_{system}", "exp_er"]
    exp_str = f"{exp_dg:.2f}" + r" $\pm$ " + f"{exp_err:.2f}"
    results_summary[system][r"Exp. $\Delta G^o_\textrm{Bind}$"] = exp_str

# Turn results summary into a dataframe, setting the first colum name to "ligand"
# Add an extra row at the top with "adaptive" spanning two columnrs and "non-adaptive" spanning two columns
results_summary_df = pd.DataFrame(results_summary).T
# Change column names to "Adaptive" and "Non-adaptive 5 ns"
results_summary_df.columns = ["Adaptive", "Non-adaptive 5 ns", "Alibay", r"Exp. $\Delta G^o_\textrm{Bind}$"]
# Change the indexes to f"Ligand {index}"
results_summary_df.index = [f"Ligand {index}" for index in results_summary_df.index]
results_summary_df.to_latex("final_analysis/results_summary.tex", escape=False)

In [ ]:
# Detailed results table

# Repeat as above, but make a much more explicit table including all contributions to the final free energy change and the
# symmetry corrections. Make the systems the columns and the rows the different contributions to the free energy change
final_dGs = final_dgs_all
results_summary_detailed = {}
for system in final_dGs:
    for time in final_dGs[system]:
        title = f"{system} {time}"
        results_summary_detailed[title] = {}
        for leg_type in final_dGs[system][time]:
            if leg_type == "dgs":
                continue
            leg_name = leg_type.split(".")[1].capitalize()
            for stage_type in final_dGs[system][time][leg_type]:
                if stage_type == "dg":
                    continue
                stage_name = stage_type.split(".")[1].capitalize()
                dgs = np.array(final_dGs[system][time][leg_type][stage_type]["dg"])
                dg_tot = np.mean(dgs)
                dg_err = get_95_ci(dgs)
                result_str = f"{dg_tot:.2f}" + r" $\pm$ " + f"{dg_err:.2f}"
                results_summary_detailed[title][f"{leg_name} {stage_name}"] = result_str
        # Add in symmetry correction and restraint correction, along with experimental results
        restraint_corr = restraint_corrections[system]
        restraint_corr_str = f"{restraint_corr:.2f}"
        results_summary_detailed[title]["Restraint Correction"] = restraint_corr_str
        symmetry_corr = symmetry_corrections[system]
        symmetry_corr_str = f"{symmetry_corr:.2f}"
        results_summary_detailed[title]["Symmetry Correction"] = symmetry_corr_str
        exp_dg = exp_dgs.loc[f"lig_{system}", "exp_dg"]
        exp_err = exp_dgs.loc[f"lig_{system}", "exp_er"]
        exp_str = f"{exp_dg:.2f}" + r" $\pm$ " + f"{exp_err:.2f}"
        results_summary_detailed[title][r"Exp. $\Delta G^o_\textrm{Bind}$"] = exp_str

results_summary_detailed_df = pd.DataFrame(results_summary_detailed).T
results_summary_detailed_df.to_latex("final_analysis/results_summary_detailed.tex", escape=False)

In [ ]:
# Summary of sampling times

df_total_costs = pd.DataFrame(total_costs)
df_total_costs.index = [f"Ligand {index}" for index in df_total_costs.index]
# Add a total row
df_total_costs.loc["Total"] = df_total_costs.sum()
# Round to integers and do not display any decimal places
df_total_costs = df_total_costs.round(0)
df_total_costs = df_total_costs.astype(int)
df_total_costs.to_latex("final_analysis/total_costs.tex", escape=False)
df_total_costs

In [ ]:
# Restraint parameters

# Double all the force constants, because of the definition in SOMD (kx rather than 0.5kx^2)
for system in restraint_dicts:
    force_constants = restraint_dicts[system]["force_constants"]
    for key in force_constants:
        force_constants[key] *= 2

# Now turn the restraints dict into a nice dataframe
restraint_df_dict = {}
for system in restraint_dicts:
    restraint_df_dict[system] = {}
    for index in restraint_dicts[system]["anchor_points"]:
        restraint_df_dict[system][index] = str(round(restraint_dicts[system]["anchor_points"][index]))
    for equil_val in restraint_dicts[system]["equilibrium_values"]:
        restraint_df_dict[system][equil_val] = f'{restraint_dicts[system]["equilibrium_values"][equil_val]:.2f}'
    for force_const in restraint_dicts[system]["force_constants"]:
        restraint_df_dict[system][force_const] = f'{restraint_dicts[system]["force_constants"][force_const]:.2f}'

replace_dict = {"r0": r"$r_0$ / $\mathrm{\AA}$", 
                "thetaA0": r"$\theta_{\mathrm{A}0}$ / $\mathrm{\AA}$", 
                "thetaB0": r"$\theta_{\mathrm{B}0}$ / Rad", 
                "phiA0": r"$\phi_{\mathrm{A}0}$ / Rad", 
                "phiB0": r"$\phi_{\mathrm{B}0}$ / Rad", 
                "phiC0": r"$\phi_{\mathrm{C}0}$ / Rad",
                "kr": r"$k_r$ / kcal mol$^{-1}$ $\mathrm{\AA}^{-2}$", 
                "kthetaA": r"$k_{\theta \mathrm{A}}$ / kcal mol$^{-1}$ $\mathrm{Rad}^{-2}$", 
                "kthetaB": r"$k_{\theta \mathrm{B}}$ / kcal mol$^{-1}$ $\mathrm{Rad}^{-2}$", 
                "kphiA": r"$k_{\phi \mathrm{A}}$ / kcal mol$^{-1}$ $\mathrm{Rad}^{-2}$", 
                "kphiB": r"$k_{\phi \mathrm{B}}$ / kcal mol$^{-1}$ $\mathrm{Rad}^{-2}$",
                "kphiC": r"$k_{\phi \mathrm{C}}$ / kcal mol$^{-1}$ $\mathrm{Rad}^{-2}$"}

# Create the dataframe
restraint_df = pd.DataFrame(restraint_df_dict)
# Replace names using the replace_dict
restraint_df = restraint_df.rename(columns=replace_dict, index=replace_dict)
# Save,making sure not to truncate the label units in the column names
with pd.option_context("max_colwidth", 1000):
    restraint_df.to_latex("final_analysis/restraint_params.tex", escape=False)

## Create Correlation Plots

In [ ]:
def compute_stats(all_results: pd.DataFrame) -> Dict[str, List[float]]:
    """
    Compute statistics for the passed results, generating 95 % C.I.s
    by bootstrapping.

    Parameters
    ----------
    all_results : pd.DataFrame
        The dataframe containing all results.

    Returns
    -------
    Dict[str, List[float]]
        A dictionary of the computed statistics, and their upper and lower
        confidence bounds.
    """

    # This will hold metric: [value, upper_err, lower_er]
    results = {"r": [], "r2": [], "mue": [], "rmse": [], "rho": [], "tau": []}

    # Get the bootstrapped results
    n_bootstrap = 10000
    bootstrapped_exp_dg, bootstrapped_calc_dg = get_bootstrapped_results(
        all_results=all_results, n_bootstrap=n_bootstrap
    )

    # For each metric, calculate i) over the actual data ii) overall bootstrapped data and extract stats
    for metric in results:
        results[metric].append(
            compute_statistic(all_results["exp_dg"], all_results["calc_dg"], metric)
        )
        bootstrapped_metric = np.zeros([n_bootstrap])
        for i in range(n_bootstrap):
            bootstrapped_metric[i] = compute_statistic(
                bootstrapped_exp_dg[i], bootstrapped_calc_dg[i], metric
            )
        percentiles = np.percentile(bootstrapped_metric, [5, 95])  # 95 % C.I.s
        results[metric].append(percentiles[0])
        results[metric].append(percentiles[1])

    return results


def get_bootstrapped_results(
    all_results: pd.DataFrame, n_bootstrap: int = 1000
) -> Tuple[np.ndarray, np.ndarray]:
    """
    Return n_bootstrap bootstrapped versions of the original experimental
    and calculated free energies.

    Parameters
    ----------
    all_results : pd.DataFrame
        The dataframe containing all results.
    n_bootstrap : int, optional, default = 1000
        Number of boostrap iterations to perform

    Returns
    -------
    boostrapped_exp_dg: np.ndarray
        The bootstrapped experimental free energy changes
    bootstrapped_calc_dg: np_ndarray
        The bootstrapped calculated free energy changes
    """
    exp_dg = all_results["exp_dg"]
    calc_dg = all_results["calc_dg"]
    exp_sem = all_results["exp_er"] / 1.96
    calc_sem = all_results["calc_er"] / 1.96

    # Check that the data passed are of the same length
    if len(exp_dg) != len(calc_dg):
        raise ValueError(
            "The lengths of the calculated and experimental free energy values must match"
        )
    n_samples = len(exp_dg)

    bootstrapped_exp_dg = np.zeros([n_bootstrap, n_samples])
    bootstrapped_calc_dg = np.zeros([n_bootstrap, n_samples])
    for i in range(n_bootstrap):
        # Ensure we use same indices for the experimental and calculated results to avoid mixing
        # results
        indices = np.random.choice(np.arange(n_samples), size=n_samples, replace=True)
        bootstrapped_exp_dg[i] = np.array(
            [np.random.normal(loc=exp_dg[i], scale=exp_sem[i]) for i in indices]
        )
        bootstrapped_calc_dg[i] = np.array(
            [np.random.normal(loc=calc_dg[i], scale=calc_sem[i]) for i in indices]
        )

    return bootstrapped_exp_dg, bootstrapped_calc_dg


def compute_statistic(exp_dg: pd.Series, calc_dg: pd.Series, statistic: str) -> float:
    """
    Compute the desired statistic for one set of experimental and
    calculated values.

    Parameters
    ----------
    exp_dg : pd.Series
        The experimental free energies
    calc_dg : pd.Series
        The calculated free energies
    statistic : str
        The desired statistic to be calculated, from "r", "mue", "rmse"
        "rho", or "tau".

    Returns
    -------
    float
        The desired statistic.
    """
    # Check that requested metric is implemented
    allowed_stats = ["r", "r2", "mue", "rmse", "rho", "tau"]
    if statistic not in allowed_stats:
        raise ValueError(
            f"Statistic must be one of {allowed_stats} but was {statistic}"
        )

    if statistic == "r":
        return stats.pearsonr(exp_dg, calc_dg)[0]
    if statistic == "r2":
        m, c, r, p, sem = stats.linregress(exp_dg, calc_dg)
        return r**2
    if statistic == "mue":
        return metrics.mean_absolute_error(exp_dg, calc_dg)
    if statistic == "rmse":
        return np.sqrt(metrics.mean_squared_error(exp_dg, calc_dg))
    if statistic == "rho":
        return stats.spearmanr(exp_dg, calc_dg)[0]
    if statistic == "tau":
        return stats.kendalltau(exp_dg, calc_dg)[0]


def plot_against_exp(
    all_results: pd.DataFrame,
    offset: bool = False,
    stats: Optional[Dict] = None,
    x_label: str = r"Experimental $\Delta G^o_{\mathrm{Bind}}$ / kcal mol$^{-1}$",
    y_label: str = r"Calculated $\Delta G^o_{\mathrm{Bind}}$ / kcal mol$^{-1}$",
) -> Tuple[plt.Figure, plt.Axes]:
    """
    Plot all results from a set of calculations against the
    experimental values.

    Parameters
    ----------
    all_results : pd.DataFrame
        A DataFrame containing the experimental and calculated
        free energy changes and errors.
    offset: bool, Optional, Default = False
        Whether the calculated absolute binding free energies have been
        offset so that the mean experimental and calculated values are the same.
    stats: Dict, Optional, Default = None
        A dictionary of statistics, obtained using analyse.analyse_set.compute_stats
    x_label: str, Optional, Default = r"Experimental $\Delta G^o_{\mathrm{Bind}}$ / kcal mol$^{-1}$"
        The label for the x-axis
    y_label: str, Optional, Default = r"Calculated $\Delta G^o_{\mathrm{Bind}}$ / kcal mol$^{-1}$"]
        The label for the y-axis

    Returns
    -------
    Tuple[plt.Figure, plt.Axes]
        The figure and axes of the plot.
    """
    # Check that the correct columns have been supplied
    required_columns = [
        "calc_base_dir",
        "exp_dg",
        "exp_er",
        "calc_cor",
        "calc_dg",
        "calc_er",
    ]
    # if list(all_results.columns) != required_column:
    #     raise ValueError(
    #         f"The experimental values file must have the columns {required_columns} but has the columns {all_results.columns}"
    #     )

    # Create the plot
    fig, ax = plt.subplots(1, 1, figsize=(6, 6))
    ax.errorbar(
        x=all_results["exp_dg"],
        y=all_results["calc_dg"],
        xerr=all_results["exp_er"],
        yerr=all_results["calc_er"],
        ls="none",
        c="black",
        capsize=2,
        lw=0.5,
    )
    ax.scatter(x=all_results["exp_dg"], y=all_results["calc_dg"], s=50, zorder=100)
    ax.set_ylim([-18, 0])
    ax.set_xlim([-18, 0])
    ax.set_aspect("equal")
    ax.set_xlabel(x_label)
    ax.set_ylabel(y_label)
    # 1 kcal mol-1
    ax.fill_between(
        x=[-25, 0],
        y2=[-24, 1],
        y1=[-26, -1],
        lw=0,
        zorder=-10,
        alpha=0.5,
        color="darkorange",
    )
    # 2 kcal mol-1
    ax.fill_between(
        x=[-25, 0],
        y2=[-23, 2],
        y1=[-27, -2],
        lw=0,
        zorder=-10,
        color="darkorange",
        alpha=0.2,
    )

    # Add text, including number of ligands and stats if supplied
    n_ligs = len(all_results["calc_dg"])
    ax.text(0.03, 0.95, f"{n_ligs} ligands", transform=ax.transAxes)
    if stats:
        stats_text = ""
        for stat, label in zip(
            ["r2", "mue", "rho", "tau"],
            ["R$^2$", "MUE", r"Spearman $\rho$", r"Kendall $\tau$"],
        ):
            stats_text += f"{label}: {stats[stat][0]:.2f}$^{{{stats[stat][2]:.2f}}}_{{{stats[stat][1]:.2f}}}$\n"
        ax.text(0.65, 0, stats_text, transform=ax.transAxes)

    return fig, ax

In [ ]:
# Get data into right format. We need a dataframe with columns "exp_dg", "calc_dg", "exp_er", "calc_er"
# Get the results for Alibay
alibay_results_for_plot = {}
for lig in overall_results_alibay:
    alibay_results_for_plot[lig] = {}
    alibay_results_for_plot[lig]["exp_dg"] = exp_dgs.loc[f"lig_{lig}", "exp_dg"]
    alibay_results_for_plot[lig]["exp_er"] = exp_dgs.loc[f"lig_{lig}", "exp_er"]
    alibay_results_for_plot[lig]["calc_dg"] = overall_results_alibay[lig]["mean_dG"]
    alibay_results_for_plot[lig]["calc_er"] = overall_results_alibay[lig]["95_ci"]
alibay_results_for_plot_df = pd.DataFrame(alibay_results_for_plot).T

# Get the results for the non-adaptive 5 ns
non_adaptive_results_for_plot = {}
for lig in final_dGs_all:
    non_adaptive_results_for_plot[lig] = {}
    non_adaptive_results_for_plot[lig]["exp_dg"] = exp_dgs.loc[f"lig_{lig}", "exp_dg"]
    non_adaptive_results_for_plot[lig]["exp_er"] = exp_dgs.loc[f"lig_{lig}", "exp_er"]
    non_adaptive_results_for_plot[lig]["calc_dg"] = np.mean(final_dGs_all[lig]["non_adaptive"]["dgs"])
    non_adaptive_results_for_plot[lig]["calc_er"] = get_95_ci(final_dGs_all[lig]["non_adaptive"]["dgs"])
    
non_adaptive_results_for_plot_df = pd.DataFrame(non_adaptive_results_for_plot).T

# Get the results for the adaptive
adaptive_results_for_plot = {}
for lig in final_dGs_all:
    adaptive_results_for_plot[lig] = {}
    adaptive_results_for_plot[lig]["exp_dg"] = exp_dgs.loc[f"lig_{lig}", "exp_dg"]
    adaptive_results_for_plot[lig]["exp_er"] = exp_dgs.loc[f"lig_{lig}", "exp_er"]
    adaptive_results_for_plot[lig]["calc_dg"] = np.mean(final_dGs_all[lig]["adaptive"]["dgs"])
    adaptive_results_for_plot[lig]["calc_er"] = get_95_ci(final_dGs_all[lig]["adaptive"]["dgs"])
adaptive_results_for_plot_df = pd.DataFrame(adaptive_results_for_plot).T

# Make a fake df to compare adaptive and non-adaptive where the "experimental" values are the non-adaptive values
# and the "calculated" values are the adaptive values
adaptive_vs_non_adaptive_results_for_plot = {}
for lig in final_dGs_all:
    adaptive_vs_non_adaptive_results_for_plot[lig] = {}
    adaptive_vs_non_adaptive_results_for_plot[lig]["exp_dg"] = non_adaptive_results_for_plot[lig]["calc_dg"]
    adaptive_vs_non_adaptive_results_for_plot[lig]["exp_er"] = non_adaptive_results_for_plot[lig]["calc_er"]
    adaptive_vs_non_adaptive_results_for_plot[lig]["calc_dg"] = adaptive_results_for_plot[lig]["calc_dg"]
    adaptive_vs_non_adaptive_results_for_plot[lig]["calc_er"] = adaptive_results_for_plot[lig]["calc_er"]
adaptive_vs_non_adaptive_results_for_plot_df = pd.DataFrame(adaptive_vs_non_adaptive_results_for_plot).T


def print_stats(stats):
    for stat in stats:
        print(f"{stat}: {stats[stat][0]:.2f} ({stats[stat][1]:.2f}, {stats[stat][2]:.2f})")

In [ ]:
# Plot for Alibay results
stats_alibay = compute_stats(alibay_results_for_plot_df)
print_stats(stats_alibay)

In [ ]:
fig, ax = plot_against_exp(alibay_results_for_plot_df, stats=stats_alibay)
fig.savefig("final_analysis/alibay_results.png", dpi=600)

In [ ]:
stats_non_adaptive = compute_stats(non_adaptive_results_for_plot_df)
print_stats(stats_non_adaptive)

In [ ]:
fig, ax = plot_against_exp(non_adaptive_results_for_plot_df, stats=stats_non_adaptive)
fig.savefig("final_analysis/non_adaptive_results.png", dpi=600)

In [ ]:
stats_adaptive = compute_stats(adaptive_results_for_plot_df)
print_stats(stats_adaptive)

In [ ]:
fig, ax = plot_against_exp(adaptive_results_for_plot_df, stats=stats_adaptive)
fig.savefig("final_analysis/adaptive_results.png", dpi=600)

In [ ]:
# Try out the comparison between the adaptive and non-adaptive runs
stats_adaptive_vs_non_adaptive = compute_stats(adaptive_vs_non_adaptive_results_for_plot_df)
print_stats(stats_adaptive_vs_non_adaptive)

In [ ]:
fig, ax = plot_against_exp(adaptive_vs_non_adaptive_results_for_plot_df, stats=stats_adaptive_vs_non_adaptive, 
                           x_label=r"Non-adaptive $\Delta G^o_{\mathrm{Bind}}$ / kcal mol$^{-1}$",
                           y_label=r"Adaptive $\Delta G^o_{\mathrm{Bind}}$ / kcal mol$^{-1}$",
                           )
fig.savefig("final_analysis/adaptive_vs_non_adaptive_results.png", dpi=600)

In [ ]:
# Make df of all stats
stats_dfs = {"Alibay": stats_alibay, "Non-adaptive": stats_non_adaptive, "Adaptive": stats_adaptive, "Adaptive vs Non-adaptive": stats_adaptive_vs_non_adaptive}
overall_stats_data = {}
for name, stats in stats_dfs.items():
    overall_stats_data[name] = {}
    for stat in stats:
        overall_stats_data[name][stat] = f"{stats[stat][0]:.2f} ({stats[stat][1]:.2f}, {stats[stat][2]:.2f})"
overall_stats_df = pd.DataFrame(overall_stats_data)

# Rename the rows to make them look nice
overall_stats_df.index = ["$r$", "$r^2$", "MUE", "RMSE", r"Spearman $\rho$", r"Kendall $\tau$"]

# Save to latex
overall_stats_df.to_latex("final_analysis/overall_stats.tex", escape=False)

## Some Statistical Tests

In [ ]:
# Check if errors are significantly different between the Alibay and non-adaptive runs
# Use paired non-parametric test
from scipy.stats import wilcoxon
res = wilcoxon(alibay_results_for_plot_df["calc_er"], non_adaptive_results_for_plot_df["calc_er"])
print(res)

In [ ]:
# Check for difference in offset between non-adaptive and Alibay results
# Get dfs of the differences to experiment
diff_alibay = alibay_results_for_plot_df["calc_dg"] - alibay_results_for_plot_df["exp_dg"]
diff_non_adaptive = non_adaptive_results_for_plot_df["calc_dg"] - non_adaptive_results_for_plot_df["exp_dg"]

# Use a Wilcoxon signed-rank test
res = wilcoxon(diff_alibay, diff_non_adaptive)
print(res)

In [ ]:
# Check for difference in offset between adaptive and non-adaptive results
# Get dfs of the differences to experiment
diff_adaptive = adaptive_results_for_plot_df["calc_dg"] - adaptive_results_for_plot_df["exp_dg"]
diff_non_adaptive = non_adaptive_results_for_plot_df["calc_dg"] - non_adaptive_results_for_plot_df["exp_dg"]

# Use a Wilcoxon signed-rank test
res = wilcoxon(diff_adaptive, diff_non_adaptive)
print(res)

In [ ]:
# Check for difference in error between adaptive and non-adaptive results
res = wilcoxon(adaptive_results_for_plot_df["calc_er"], non_adaptive_results_for_plot_df["calc_er"])
print(res)

In [ ]:
costs

## Lambda Windows

In [ ]:
# For every stage for every system, plot the lambda values vs lambda index and show a dotted line on x = y
fig, axs = plt.subplots(3, 9, figsize = (28, 9))
for i, lig in enumerate(ligs):
    for j, leg in enumerate(lam_vals[lig]):
        for k, stage in enumerate(lam_vals[lig]["LegType.BOUND"]):
            if k == 0 and leg == "LegType.FREE":
                continue
            stage_str = stage.split(".")[1].lower().capitalize()
            leg_str = leg.split(".")[1].lower().capitalize()
            lambda_idx = list(range(len(lam_vals[lig][leg][stage])))
            linear = np.linspace(0, 1, len(lambda_idx))
            lam_vals_local = lam_vals[lig][leg][stage]
            axs[k, i].plot(lambda_idx, lam_vals_local, label = leg_str, marker = "o")
            axs[k, i].plot(lambda_idx, linear, linestyle = "--", color = "black")
            axs[k, i].set_title(f"{lig} {stage_str}")
            axs[k, i].set_xlabel("$\lambda$ index")
            axs[k, i].set_ylabel("$\lambda$ value")
            # Add text stating the number of windows
            offset = 0.05 if leg == "LegType.FREE" else 0.1
            axs[k, i].text(0.6, offset, f"No. windows {leg_str}: {len(lam_vals_local)}", transform = axs[k, i].transAxes, horizontalalignment = "center", verticalalignment = "center")

# Add a legend to the figure, off to the right hand side of all the plots
axs[1,8].legend(bbox_to_anchor = (1.05, 0.5), loc = "center left", borderaxespad = 0)
fig.tight_layout()
fig.savefig("final_analysis/lambda_values.png", dpi = 600, bbox_inches = "tight")

# Sampling Time Allocations

In [ ]:
fig, axs = plt.subplots(5, 9, figsize = (30, 17), dpi = 600)
for i, lig in enumerate(ligs):
    for j, leg in enumerate(sampling_times[lig]):
        for k, stage in enumerate(sampling_times[lig][leg]):
            stage_str = stage
            leg_str = leg
            times = sampling_times[lig][leg][stage]["times"]
            times = np.array(times) * costs[lig][leg] * REF_COST
            equil_times = sampling_times[lig][leg][stage]["equil_times"]
            equil_times = np.array(equil_times) * costs[lig][leg] * REF_COST
            lam_vals_local = sampling_times[lig][leg][stage]["lam_vals"]
            # Bar plots for sampling times
            ax = axs[k + 3*j, i]
            # Get reasonable width for bars
            width = 0.6 / len(lam_vals_local)
            ax.bar(lam_vals_local, times, label = "Total sampling time", edgecolor = "black", width = width)
            # Plot the equilibration times
            ax.bar(lam_vals_local, equil_times, label = "Equilibration time", edgecolor = "black", width = width, hatch = "///////")
            ax.set_title(f"{lig} {leg_str} {stage_str}")
            ax.set_xlabel("$\lambda$")
            ax.set_ylabel("GPU Hours")

# Create a legend for the figure to the right of all the plots
axs[2,8].legend(bbox_to_anchor = (1.05, 0.5), loc = "center left", borderaxespad = 0)
fig.tight_layout()
fig.savefig("final_analysis/sampling_times.png", dpi = 600, bbox_inches = "tight")

In [ ]:
# Plot bar plot of costs
costs_df = pd.DataFrame(costs)
# Nice black outline round bars
costs_df.plot.bar(figsize = (10, 5), rot = 0, xlabel = "Leg", ylabel = "Relative cost", title = "Relative cost of each leg", edgecolor = "black")
# Save figure
plt.savefig("final_analysis/costs.png", dpi = 600, bbox_inches = "tight")

In [ ]:
# All similar, as expected, so just get the mean cost for each leg
costs_df.mean(axis = 1)

In [ ]:
# Create a coarser summary of sampling time allocation - just show the total sampling time for each stage for each sytem.
# Put everything on a single bar plot. Use the code above as a starting point.

fig, ax = plt.subplots(figsize=(8, 4), dpi=600)
x = np.arange(len(sampling_times))
width = 0.2
# Plot bound and free next to each other
for i, stage in enumerate(sampling_times["2"]["bound"]):
    # Get single colour
    color = ax._get_lines.get_next_color()
    tot_times_bound = [np.sum(sampling_times[system]["bound"][stage]["times"])*costs[system]["bound"]*REF_COST for system in sampling_times]
    ax.bar(x + (i * width), tot_times_bound, width, label=f"Bound {stage}", edgecolor="k", alpha=1, color=color)
    if stage != "restrain":
        tot_times_free = [np.sum(sampling_times[system]["free"][stage]["times"])*costs[system]["free"]*REF_COST for system in sampling_times]
        ax.bar(x + (i * width), tot_times_free, width, label=f"Free {stage}", edgecolor="k", alpha=1, color=color, hatch="///////")

ax.set_xticks(x + width)
x_labels = []
for system in sampling_times:
    x_labels.append(system)
ax.set_xticklabels(x_labels)
ax.set_ylabel("Total Sampling Time / GPU Hours")
# Put label off to right of plot
ax.legend(bbox_to_anchor=(1.03, 0.7))
plt.tight_layout()
fig.savefig("final_analysis/sampling_times_summary.png", bbox_inches="tight", dpi=600)


In [ ]:
fig, ax = plt.subplots(figsize=(8, 4), dpi=600)
x = np.arange(len(sampling_times))
width = 0.2
# Plot bound and free next to each other
for i, stage in enumerate(sampling_times["2"]["bound"]):
    # Get single colour
    n_windows = np.array([len(sampling_times[system]["bound"][stage]["times"]) for system in sampling_times])
    color = ax._get_lines.get_next_color()
    tot_times_bound = [np.sum(sampling_times[system]["bound"][stage]["times"])*costs[system]["bound"]*REF_COST for system in sampling_times]
    tot_times_bound_per_window = tot_times_bound / n_windows
    ax.bar(x + (i * width), tot_times_bound_per_window, width, label=f"Bound {stage}", edgecolor="k", alpha=1, color=color)
    if stage != "restrain":
        tot_times_free = [np.sum(sampling_times[system]["free"][stage]["times"])*costs[system]["free"]*REF_COST for system in sampling_times]
        tot_times_free_per_window = tot_times_free / n_windows
        ax.bar(x + (i * width), tot_times_free_per_window, width, label=f"Free {stage}", edgecolor="k", alpha=1, color=color, hatch="///////")

ax.set_xticks(x + width)
x_labels = []
for system in sampling_times:
    x_labels.append(system)
ax.set_xticklabels(x_labels)
ax.set_ylabel("Total Sampling Time per Window / GPU Hours")
# Put label off to right of plot
ax.legend(bbox_to_anchor=(1.03, 0.7))
plt.tight_layout()
fig.savefig("final_analysis/sampling_times_summary_per_window.png", bbox_inches="tight", dpi=600)


## Plots of Free Energy vs Sampling Time

In [ ]:
sampling_times_non_adaptive = sampling_times_nonadapt

def plot_dgs_conv(ax: plt.axes, dgs: np.ndarray,times: np.ndarray, system: str, label: str) -> None:
    # Plot the mean free energy
    mean_free_energy = np.mean(dgs, axis=0)
    ax.plot(times, mean_free_energy, label=label)
    # Calculate the 95 % CI with scipy
    conf_int = (
    t.interval(
        0.95,
        len(dgs) - 1,
        mean_free_energy,
        scale=sem(dgs),
    )[1]
    - mean_free_energy
    )  # 95 % C.I.
    # Fill between the upper and lower bounds of the 95 % CI
    ax.fill_between(
        times,
        mean_free_energy - conf_int,
        mean_free_energy + conf_int,
        alpha=0.2,
    )
    # Label the plot
    ax.set_xlabel("GPU Hours")
    ax.set_ylabel(r"$\Delta G$ / kcal mol$^{-1}$")
    ax.set_title(f"Lig {system}")
    ax.legend()
    # Ensure that the bottom of the scale is 0
    plt.tight_layout()

def plot_cis_conv_all(dg_dict_nonadapt: Dict, dg_dict_adapt: Dict, scale:bool=False) -> Tuple[plt.Figure, plt.Axes]:
    """Plot the change in the 95 % CIs for each system"""
    n_systems = len(dg_dict_nonadapt)
    n_cols = 5
    n_rows = int(np.ceil(n_systems / n_cols))
    fig, axs = plt.subplots(n_rows, n_cols, figsize=(20, 8))
    axs = axs.flatten()

    for dict_type, dg_dict in {"Adaptive": dg_dict_adapt, "Non-Adaptive": dg_dict_nonadapt}.items():
        samp_times = sampling_times if dict_type == "Adaptive" else sampling_times_non_adaptive
        for i, lig in enumerate(ligs):
            overall_dgs = np.zeros_like(dg_dict[lig]["LegType.BOUND"]["StageType.DISCHARGE"]["dgs"])
            overall_times = np.zeros(overall_dgs.shape[1])
            for leg_name, leg in dg_dict[lig].items():
                leg_label = leg_name.split(".")[1].lower()
                dg_multiplier = 1 if leg_label == "free" else -1
                cost = costs[lig][leg_label] * REF_COST 
                for stage_name in leg:
                    stage_label = stage_name.split(".")[1].lower()
                    stage_dgs = leg[stage_name]["dgs"] * dg_multiplier
                    tot_time = np.sum(samp_times[lig][leg_label][stage_label]["times"]) * costs[lig][leg_label] * REF_COST
                    stage_times = np.array(leg[stage_name]["fracts"]) * tot_time
                    overall_dgs += stage_dgs
                    overall_times += stage_times
            
            # Add the restraint corrections
            overall_dgs -= restraint_corrections[lig]

            plot_dgs_conv(axs[i], overall_dgs, overall_times, lig, dict_type)

    # Delete unused axes
    for i in range(n_systems, n_rows * n_cols):
        fig.delaxes(axs[i])
        
    return fig, axs


fig, axs = plot_cis_conv_all(dgs_conv_adaptive_nonequil, dgs_conv_non_adaptive_nonequil)
fig.savefig("final_analysis/cyclod_dgs_conv_overall.png", dpi=300)